In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [2]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [3]:
v = t.T[[
    'R00200','R00268','R00341','R00342','R00351','R00361','R00658','R01061','R01070','R01082','R01196',
    'R01197','R01325','R01512','R01518','R01786','R01899','R01900','R02164','R02740','R04779','R10343'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)

v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [12]:
df = v.T
df['Glucose'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose']['C00267'] = 1

# Oxophos
# df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
# df['Oxophos']['C00004'] = -2
# df['Oxophos']['C00007'] = -1
# df['Oxophos']['C00008'] = -5
# df['Oxophos']['C00002'] = 5
# df['Oxophos']['C00003'] = 2
# df['Oxophos']['C00011'] = 1
# Oxygen

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['Biomass'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Biomass']['C00036'] = -1

df = df.drop([
    # H2O   # Phosphate # CO2 #Glycerone phosphate # H+
    'C00001','C00009','C00011','C00111','C00080',
    # quinon  # quinon  # Acetate
    'C00138','C00139','C00033'
])

In [13]:
x = []
for i in df.columns:
    if  (i=='Biomass') or (i=='Oxophos'):
        x.append(LpVariable(i, lowBound=0,upBound=100, cat='Continuous'))
        
    elif (i=='Glucose'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='Oxygen'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='R00341') or (i=='R00200'):
        x.append(LpVariable(i, lowBound=-100, upBound=0, cat='Continuous'))
    
    else:
        x.append(LpVariable(i, lowBound=-100, upBound=100, cat='Continuous'))

In [14]:
m = LpProblem(sense=LpMaximize)
m += x[-1]

for i in np.dot(df, x):
    m += i == 0

In [15]:
m

NoName:
MAXIMIZE
1*Biomass + 0
SUBJECT TO
_C1: - R00200 - R00341 - R01512 - R01786 - R04779 = 0

_C2: - NADP_Exchange - R00342 - R01061 = 0

_C3: NADP_Exchange + R00342 + R01061 = 0

_C4: - NADP_Exchange + R01899 = 0

_C5: NADP_Exchange - R01899 = 0

_C6: R00200 + R00341 + R01512 + R01786 + R04779 = 0

_C7: - R00351 + R01196 + R01197 = 0

_C8: - R00200 + R01196 = 0

_C9: R00351 - R01196 + R10343 = 0

_C10: R00268 + R01197 = 0

_C11: - Biomass - R00341 + R00342 + R00351 + R00361 = 0

_C12: - R02164 + R10343 = 0

_C13: R00200 + R00341 + R00658 = 0

_C14: - R01197 - R10343 = 0

_C15: - R01061 + R01070 = 0

_C16: R01082 + R02164 = 0

_C17: - R00342 - R00361 - R01082 = 0

_C18: - R00351 - R01325 = 0

_C19: - R01512 + R01518 = 0

_C20: R01061 + R01512 = 0

_C21: Glucose - R01786 = 0

_C22: - R01899 - R01900 = 0

_C23: R01325 + R01900 = 0

_C24: - R00658 - R01518 = 0

_C25: R01786 - R02740 = 0

_C26: R02740 - R04779 = 0

_C27: - R01070 + R04779 = 0

_C28: - R00268 + R01899 = 0

_C29: - R00361

In [16]:
status = m.solve()
print(LpStatus[status])

Optimal


In [20]:
y = []
for i in x:
    y.append(i.value())

In [21]:
pd.DataFrame(y, index=x)

,0
R00200,0.000000
R00268,-33.333333
R00341,-100.000000
R00342,-66.666667
R00351,33.333333
R00361,33.333333
R00658,100.000000
R01061,100.000000
R01070,100.000000
R01082,33.333333
